In [1]:
from typing import List, Dict

# Cashu Mints

6 NUTS to implement. 

Things to do:

- [x] Generate pubkeys from a seed
    - different pubkeys for different amounts??
    - only one active key set
- [ ] Signing of tokens
- [ ] Verify tokens
- [x] Calculate keyset Ids
- [ ] make it so that wallet can request to mint
        - store hash with invoice for lookup later
- [ ] mint tokens
    - verify `BlindedMessage`s
    - create promises
- [ ] melt tokens - when a token is spent we need to invalidate it
- [ ] split tokens - when a request to split is made, invalidate previous tokens and calculate value of new one
    

## [Generating a Keyset](https://github.com/cashubtc/nuts/blob/main/02.md#mints-generating-a-keyset)

```python
for i in range(MAX_ORDER):
	k_i = HASH_SHA256(s + D + i)[:32]
 ```

`s` - seed

`k_i` - priv key

`i` - index of amount value

`MAX_ORDER` - max token values supported 
   - Typically, `MAX_ORDER = 64`

`D` - derivation path

Check this out: https://github.com/cashubtc/nutshell/blob/main/cashu/core/crypto/keys.py#L18C1-L18C1 

In [2]:
from hashlib import sha256

def generate_private_keyset(seed, MAX_ORDER, derivation_path):
    to_hash = [(seed + derivation_path + str(i)).encode() for i in range(MAX_ORDER)]
    keys = [sha256(x).digest()[:32] for x in to_hash]
    return keys

In [3]:
seed = "johny_apple"
MAX_ORDER = 8
derivation_path = "/0/0/0/0"

keys = generate_private_keyset(seed, MAX_ORDER, derivation_path)

[k.hex() for k in keys]


['ab4af145c55ea0a004725df14435c6d1ced8fd8541c2fa01248263336a682035',
 'b4ffa1f7282866458fe754c4fec907c7efd03119964be20ea3571eb9335e217a',
 'c0401353920db3376f97a78556d4d4352207273e8eabc0c3b958040351dd2d74',
 'b8e6c9b7c33e2e4941a8701f0c66ed9ccaf191c5ffed595696db45c20b9126ce',
 'fb146d546dba6dfd4151ef44e9a00ff4704ce90d80ce6d87ec8bb4b56bff386d',
 '1fa7dd758569da6b518c3a1268f9bc7a613281f535814c65e7b4a8f565b2600e',
 '900ec67c4c57c93bf7795b8b08d5d8620cf93883f209a55b2e3100eca59a5e35',
 '804dc5af2e5a71b58a8463ad55beb28e9258fd0e346fcae7d3e0200ce4513c3e']

## [Public Keyset](https://github.com/cashubtc/nuts/blob/main/02.md#nut-02-keysets-and-keyset-id)
A keyset is a set of public keys that the mint Bob generates and shares with its users. The mint MUST use the compressed public key format. The keyset refers to the set of public keys that each correspond to the amount values that the mint supports (e.g. 1, 2, 4, 8, ...) respectively.

In [4]:
from coincurve import PublicKey

def get_keyset(priv_keys: List[str], supported_values: List[int]):
    pubkeys = [PublicKey.from_secret(k).format().hex() for k in priv_keys]
    keyset = {value: pubkeys[value - 1] for value in supported_values if value - 1 < len(pubkeys)}
    return keyset

In [5]:
public_keyset = get_keyset(keys, [1,2,4,8])
public_keyset

{1: '02c1bdb050a07094e99a4295b9698b53eac9725124fb8d2bade159d6c42373bcdc',
 2: '029fb6c2ff337f13cfeaa5cca4a5b9fad0299facd18b3f3f2b70b38655162ace31',
 4: '03cac6b5f54406176b34438781dc08b35ca254bf1e1b024edc65fd8aa5b3ccdb27',
 8: '03ed86baf1c5003b66a2ca73cb5224cd71cc0d736d24b1b4d68785e8b9be06af8d'}

## [Deriving Keyset ID](https://github.com/cashubtc/nuts/blob/main/02.md#keyset-id)

1.  sort keyset by amount
2.  concatenate all (sorted) public keys to one string
3.  HASH_SHA256 the concatenated public keys
4.  take the first 12 characters of the hash

In [6]:
from base64 import b64encode

def derive_keyset_id(pub_keyset: Dict[int, PublicKey]):
    sorted_by_amount = dict(sorted(pub_keyset.items()))
    pubkeys_concat = "".join([p for _, p in sorted_by_amount.items()])
    hashed_pubkeys = sha256(pubkeys_concat.encode("utf-8")).digest()
    return b64encode(hashed_pubkeys).decode()[:12]

def urlsafe_to_keyset_id(keyset_id_url_safe):
    return keyset_id_url_safe.replace('_', '/').replace('+', '-')

def keyset_id_to_urlsafe(keyset_id):
    return keyset_id.replace('/', '_').replace('-', '+')
    

In [7]:
keyset_id = derive_keyset_id(public_keyset)
print("KeysetID: ", keyset_id)

url_safe_keyset_id = keyset_id_to_urlsafe(keyset_id)
print ("URL Safe: ", url_safe_keyset_id)

assert keyset_id == urlsafe_to_keyset_id(url_safe_keyset_id)

KeysetID:  ioID1wgpCeGz
URL Safe:  ioID1wgpCeGz


## Questions

- What do we use as the derivation path `D` when generating a keyset? Can it be arbitrary?

- do we store each priv key? Or just the derivation path and keyset ID then generate keys each time for signing? This way we only store the seed?
- Does the mint have one single pubkey and single prive other than masterkey??
  - https://github.com/cashubtc/nutshell/blob/main/cashu/core/crypto/keys.py#L35
  - https://github.com/cashubtc/nutshell/blob/main/cashu/mint/ledger.py#L58

# [Blind Diffie-Hellmann key exchange (BDHKE)](https://github.com/cashubtc/nuts/blob/main/00.md#blind-diffie-hellmann-key-exchange-bdhke) 

## Bob (mint)

- `k` private key of mint (one for each amount)
- `K` public key of mint
- `Q` promise (blinded signature)

## Alice (user)

- `x` random string (secret message), corresponds to point `Y` on curve
- `r` private key (blinding factor)
- `T` blinded message
- `Z` proof (unblinded signature)

 ---

- Mint `Bob` publishes public key `K = kG` 
- `Alice` picks secret `x` and computes `Y = hash_to_curve(x)`
- `Alice` sends to `Bob`: `B_ = Y + rG` with `r` being a random blinding factor (**blinding**)
- `Bob` sends back to `Alice` blinded key: `C_ = kB_` (these two steps are the DH key exchange) (**signing**)
- `Alice` can calculate the unblinded key as `C_ - rK = kY + krG - krG = kY = C` (**unblinding**)
- Alice can take the pair `(x, C)` as a token and can send it to `Carol`.
- `Carol` can send `(x, C)` to `Bob` who then checks that `k*hash_to_curve(x) == C` (**verification**), and if so treats it as a valid spend of a token, adding `x`  to the list of spent secrets.

Who knows what??
Carol: K, x, Y, B_, r, C_

Bob: K, k, B_, C_

Who doesn't know what??

Carol: k

Bob: x, r, Y

In [10]:
K = public_keyset[1]
k = keys[0]

In [11]:
def hash_to_curve(x_bytes):
    # Hash the secret using SHA-256
    hash_value = sha256(x_bytes).digest()
    # Create a public key Y from the hashed secret
    Y = PublicKey.from_secret(hash_value)
    return Y

In [12]:
x_secret = b'a secret'

In [13]:
Y = hash_to_curve(x_secret)
Y

**Alice sends to Bob:** `B_ = Y + rG`

In [14]:
from coincurve import PrivateKey
r = PrivateKey()
R = r.public_key
B_ = PublicKey.combine_keys([Y, R])
B_

**Bob send to Alice:** `C_ = kB_`

In [15]:
# priv key that corresponds to token value `1`
k1 = keys[0]
C_ = B_.multiply(k1)
C_

**Alice:**   `C_ - rK = kY + krG - krG = kY = C`

In [16]:
# we are using the pub key for token value `1`
K = public_keyset[1]

# order of the finite field
p = 115792089237316195423570985008687907853269984665640564039457584007908834671663

rK = PublicKey(bytes.fromhex(K)).multiply(r.secret)

x,y = rK.point()
neg_rK = PublicKey.from_point(x, (p - y) % p)

# C = C_ - rK
C = PublicKey.combine_keys([C_, neg_rK])

# kY should equal C
kY = Y.multiply(k)
kY == C

True

Now the token is `(x, c)`

In [17]:
token = (x_secret, C)
token

(b'a secret', <coincurve.keys.PublicKey at 0x7ff9c00b9390>)

Carol can now redeem with Bob.

Bob verifies the token with:

`k*hash_to_curve(x) == C`

In [18]:
x_secret, C = token
C == hash_to_curve(x_secret).multiply(k)

True

If valid spend the **add `x_secret` to list of spent secrets**

# [Requesting a Mint](https://github.com/cashubtc/nuts/blob/main/03.md)
1. Wallet sends `GET /mint?amount=<amount_sat>`
2. Mint responds with invoice and hash:

```json
{
  "pr": "lnbc100n1p3kdrv5sp5lpdxzghe5j67q...",
  "hash": "67d1d9ea6ada225c115418671b64a..."
}
```
3. Wallet pays invoice
4. Wallet requests tokens: `POST /mint?hash=<hash>` and `BlindedMessages` in request body
5.  d

#### `BlindedMessage` (outputs)
```json
{
  "amount": int,
  "B_": hex_str
}
```

`BlindedMessages` is an array of `BlindedMessage`s

**hash must be unique and stored with amount_sat**


##### Verify `BlindedMessages`
- `amount`s must add to a maximum of `<amount_sat>`
   - find amouunt sat with the `hash`
- invoice must be paid
- `BlindedMessages` are valid

#### `PostMintResponse`

If request for token is valid, then mint responds with `BlindedSignature`s (`C_`)

```json
{
"promises":
  [
    {
    "id": "DSAl9nvvyfva",
    "amount": 2,
    "C_": "03e61daa438fc7bcc53f6920ec6c8c357c24094fb04c1fc60e2606df4910b21ffb"
    },
    {
    "id": "DSAl9nvvyfva",
    "amount": 8,
    "C_": "03fd4ce5a16b65576145949e6f99f445f8249fee17c606b688b504a849cdc452de"
    },
  ]
}
```


#### Unblinfing Signatures

Once the wallet receives a response from mint, then unblind to create `Proofs`.

##### `Proofs`
```json
{
"proofs" : 
  [
    {
    "id": "DSAl9nvvyfva", //keyset id
    "amount": 2, //token amount value
    "secret": "S+tDfc1Lfsrb06zaRdVTed6Izg", // secret used by wallet in the signing process
    "C": "0242b0fb43804d8ba9a64ceef249ad7a60f42c15fe6d4907238b05e857527832a3" // unblinded sig
    },
    {
    ...
    }
  ]
}
```

## Questions
- What does the mint need to store? Just the secrets that have been spent?
    - If the mint signs a token where that secret is hidden, then the mint know that they issued the token, but there is one secret for one token, so if secret is spent thats all we need to know... right?? right??
    - real question: one secret for one token??

# [Melting Tokens](https://github.com/cashubtc/nuts/blob/main/05.md#nut-05-melting-tokens)
Opposite of minting

1. wallet gets maximum fees `POST /fees` with ln invoice
2. mint responds with fees
3. wallet send `Proofs` along with a lightning invoice `POST /melt` (mint expexts `Proofs` of at least `total_amount = amount + fee_reserve`
4. mint responds with:

```json
{
"paid": true,
"preimage": "da225c115418671b64a67d1d9ea6a..."
}
```

Once paid, **mint should mark all secrets from tokens as spent**

# [Splitting Tokens](https://github.com/cashubtc/nuts/blob/main/06.md#nut-06-split-tokens)

- Involves `Proofs` and `BlindedMessages` in the request to split
- Mint verifies, then invalidates `Proofs` --> then issues new promises `BlindedSignatures`


- Wallet requests split `POST /split` with `Proofs` and `BlindedMessages` in request body
- Mint responds with `BlindedSignatures` if all good

Nice function for getting split amounts https://github.com/cashubtc/nutshell/blob/main/cashu/core/split.py

# What is needed to run a mint??

- ability to blind sign 
- ability to verify that I signed 
- database to keep track of things so we don't lose money and be sad
  - spent secrets
  - `<hash>:<amount_sat>` and maybe an invoice for the minting process??
- API for interacting with wallets